In [1]:
import pandas as pd 
import os 
from dotenv import load_dotenv
load_dotenv()

os.chdir("..")


In [2]:
# file for category mapping
cat_file = "./data/Rocks_N_Ropes_Chat_2022-06-21v2.csv"
chat_file = "./data/RnR Chat 2021-11-11.csv"


In [5]:
cat_df = pd.read_csv(cat_file,sep=",")
chat_df = pd.read_csv(chat_file,sep=",")

display(cat_df.head())
display(chat_df.head())

,Category,Topic,Keyword
0,Emotion,Mad,I'm _ mad
1,Emotion,Mad,sucks
2,Emotion,Mad,I hate
3,Emotion,Mad,not _ happy
4,Emotion,Mad,pisses me off


,CustomerID,contact_type,site_down,sale_accept,site_down_sentiment,product_name,product_family,order_identifier,carrier,customer_state,...,c_address_response,a_shipping_prompt_2,c_shipping_response_2,a_order_confirmation,a_return_instructions,c_customer_email,a_label_notice,a_agent_closing_start,c_customer_closing_response,a_agent_closing_wrap
0,1966,Sales Order,0,1,9,Fiction jacket,clothing,email,FedEx,Ohio,...,7 Mitchell Parkway Columbus Ohio 43231,Use the same address for mailing?,"Yes, thanks",You will receive a confirmation email in the n...,9,9,9,Is that all you needed today?,"No, and I won't be shopping here again","Well, thanks for contacting R and R"
1,1403,Sales Order,0,1,9,UltraSham heavy jacket,clothing,order number,USPS,North Carolina,...,491 Thierer Alley Charlotte North Carolina 28210,Use the same address for shipping?,Yes,You will receive a confirmation email in the n...,9,9,9,Is there anything else I can do to assist you?,"No, thanks",Thank you for contacting Rocks and Ropes
2,2402,Return,0,0,9,Mountbank vest,clothing,order number,UPS,Florida,...,9,9,9,9,"To process your return, I'm going to send you ...",rbiggins97@ocn.ne.jp,Thank you for that. You should receive the ret...,Do you need any other assistance?,"No, thanks for doing such a great job","Well, thank you for contacting RnR"
3,1209,Return,0,1,9,Fiction jacket,clothing,order number,UPS,California,...,9,9,9,9,"To process your return, I'm going to send you ...",msumsion7c@liveinternet.ru,Thank you for that. You should receive the ret...,Do you need any other assistance?,"Yes, but you can't fix it",9
4,2447,Sales Order,0,1,9,Charlatan sleeping bag,sleeping bag,order number,UPS,Indiana,...,46 Melrose Junction Indianapolis Indiana 46266,Would you like us to ship to that address too?,They are,You will receive a confirmation email in the n...,9,9,9,Can I help with anything else?,"No, I appreciated your assistance",Thanks for contacting Rocks and Ropes


In [15]:
# get relevatn columns to rebuild the transcript 
chat_cols = chat_df.columns
trans_cols = [col for col in chat_cols if ("a_" in col) or ("c_" in col)]

# build the full text ? from the row ?  kind of slow but easier process to write
full_array = []
for i, row in chat_df.iterrows():
    text = []
    for col in trans_cols:
        if row[col] !="9": 
            text.append(row[col])
        else:
            text.append("")
    
    full_array.append(" ".join(text)) # join it back

chat_df["transcripts"] = full_array # processed in serial order should align
chat_df.head()

# save off so jeff doesn't have to reprocess this in the future
chat_df.to_csv("./data/rnr_chat_w_transcript.csv", sep=",")


# Assumptions

exact match except some special characters are ignored ( . , !, etc.)  2. More than 1 Topic can be applied to a conversation if multiple Topics result in keyword hits. 3. The results are not applied based on Frequency. 4) the scan is performed across the entire conversation (transcript) and the results are reported at the conversation level. The app does highlight all the instances of the keyword / topic hits within the conversation when reviewing the transcript


# process flow 
- build transcript based on columns 
- for each transcript
    - preprocess 
    - tag topic if initial key word match is found w

- append transcript 
- create tag table 
    - transcript Id 
    - chat_id 
    - tag 
    (eessentially for sake of easy processing explode the df on tag array so can process in excel more easily for tying metrics back to CXI)
    
